# PyTorch를 활용한 영역 분할

목표

- COCO 사전학습 Mask R-CNN 모델로 PyTorch 추론

- 추론 결과 시각화 (Bounding Box + Mask)

- ONNX로 변환

- ONNX Runtime으로 추론

- PyTorch vs ONNX 추론 결과 시각화 및 시간 비교

## 1. 필수 라이브러리 설치 및 불러오기

In [ ]:
!pip install onnx onnxruntime --quiet

In [ ]:
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import time
import onnx
import onnxruntime as ort

from PIL import Image
import requests
from io import BytesIO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

## 2. 사전학습 Mask R-CNN 모델 불러오기 및 테스트 이미지 준비

In [ ]:
!wget https://raw.githubusercontent.com/JinFree/OpenCV_for_DeepLearning/3e8b5613ff76389b78d22c35b4e30678a887aca7/Data/image_01.png

In [ ]:
# 사전학습된 COCO 기반 Mask R-CNN
model = maskrcnn_resnet50_fpn(weights="DEFAULT")
model.eval().to(device)

# 테스트 이미지 불러오기
img = cv2.imread('/content/image_01.png')
img_resize = cv2.resize(img, (224, 224))
img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)
img_normalized = img_rgb.astype(np.float32) / 255.0
img_chw = np.transpose(img_normalized, (2, 0, 1))
img_tensor = np.expand_dims(img_chw, axis=0)
img_tensor = torch.from_numpy(img_tensor).to(device)

# 모델 입력은 리스트로 전달
with torch.no_grad():
    output = model(img_tensor)[0]

## 3. PyTorch 추론 결과 시각화 (Bounding Box + Mask)

In [ ]:
# COCO 클래스 목록
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# 시각화 함수
def show_maskrcnn_result(img, output, score_threshold=0.8):
    img_np = np.array(img).copy()
    masks = output['masks'].cpu().numpy()
    boxes = output['boxes'].cpu().numpy()
    labels = output['labels'].cpu().numpy()
    scores = output['scores'].cpu().numpy()

    for i in range(len(masks)):
        if scores[i] < score_threshold:
            continue
        mask = masks[i, 0] > 0.5
        color = np.random.randint(0, 255, (3,), dtype=np.uint8)
        img_np[mask] = img_np[mask] * 0.5 + color * 0.5

        x1, y1, x2, y2 = boxes[i]
        cv2.rectangle(img_np, (int(x1), int(y1)), (int(x2), int(y2)), color.tolist(), 2)
        cv2.putText(img_np, f"{COCO_INSTANCE_CATEGORY_NAMES[labels[i]]}: {scores[i]:.2f}",
                    (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)

    plt.figure(figsize=(10, 8))
    plt.imshow(img_np)
    plt.axis("off")
    plt.show()

# 결과 보기
img = cv2.imread('/content/image_01.png')
img_resize = cv2.resize(img, (224, 224))
img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)
show_maskrcnn_result(img_rgb, output)


## 4. ONNX로 변환 (마스크 추론 포함)

In [ ]:
# 모델을 CPU로 이동 후 dummy input 생성
model.cpu()
dummy_input = torch.randn(1, 3, 224, 224)

onnx_path = "maskrcnn.onnx"

torch.onnx.export(model,
                  dummy_input,
                  onnx_path,
                  input_names=['input'],
                  output_names=['boxes', 'labels', 'scores', 'masks'],
                  opset_version=11,
                  do_constant_folding=True)

print("ONNX 모델로 저장 완료:", onnx_path)


## 5. ONNX Runtime으로 추론하기

Note: PyTorch의 Mask R-CNN은 ONNX 변환 시 복잡한 마스크 추론 경로를 포함하므로, 일부 버전에서는 마스크 추론이 제대로 되지 않을 수 있습니다. 아래는 박스/레이블/스코어 중심의 추론 예입니다.

In [ ]:
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])


img = cv2.imread('/content/image_01.png')
img_resize = cv2.resize(img, (224, 224))
img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)
img_normalized = img_rgb.astype(np.float32) / 255.0
img_chw = np.transpose(img_normalized, (2, 0, 1))
img_tensor = np.expand_dims(img_chw, axis=0)

# ONNX 추론
outputs = session.run(None, {"input": img_tensor})
onnx_boxes, onnx_labels, onnx_scores, onnx_masks = outputs

print("ONNX 결과 박스 수:", len(onnx_boxes))

ONNX Runtime 추론 결과에서 마스크 시각화하기

- ONNX로 변환된 Mask R-CNN 모델의 추론 결과는 일반적으로 다음과 같은 네 가지 출력을 포함합니다:
  - boxes: 탐지된 객체의 경계 상자 (N, 4)
  - labels: 각 객체의 클래스 레이블 (N,)
  - scores: 각 탐지의 신뢰도 점수 (N,)
  - masks: 각 객체의 마스크 (N, 1, H, W)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def visualize_masks(image, boxes, masks, labels, scores, class_names, score_threshold=0.5):
    """
    이미지에 마스크와 경계 상자를 시각화합니다.

    Parameters:
    - image: 원본 이미지 (numpy array, H x W x 3)
    - boxes: 경계 상자 (N x 4)
    - masks: 마스크 (N x 1 x H x W)
    - labels: 클래스 레이블 (N,)
    - scores: 신뢰도 점수 (N,)
    - class_names: 클래스 이름 리스트
    - score_threshold: 시각화할 최소 신뢰도 점수
    """
    image = image.copy()
    for i in range(len(scores)):
        if scores[i] < score_threshold:
            continue

        mask = masks[i, 0]
        mask = (mask > 0.5).astype(np.uint8)

        # 무작위 색상 생성
        color = np.random.randint(0, 256, (3,), dtype=np.uint8)

        # 마스크 적용
        colored_mask = np.zeros_like(image, dtype=np.uint8)
        for c in range(3):
            colored_mask[:, :, c] = color[c] * mask
        image = cv2.addWeighted(image, 1.0, colored_mask, 0.5, 0)

        # 경계 상자 그리기
        x1, y1, x2, y2 = boxes[i].astype(int)
        cv2.rectangle(image, (x1, y1), (x2, y2), color.tolist(), 2)

        # 클래스 이름 및 점수 표시
        label_text = f"{class_names[labels[i]]}: {scores[i]:.2f}"
        cv2.putText(image, label_text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [ ]:
# ONNX Runtime을 사용하여 추론
outputs = session.run(None, {"input": img_tensor})
onnx_boxes, onnx_labels, onnx_scores, onnx_masks = outputs

# 클래스 이름 리스트 (예: COCO 데이터셋)
class_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# 이미지 시각화
visualize_masks(img_resize, onnx_boxes, onnx_masks, onnx_labels.astype(int), onnx_scores, class_names, score_threshold=0.5)


## 6. 추론 속도 비교 (PyTorch vs ONNXRuntime)

In [ ]:
# tensor
img = cv2.imread('/content/image_01.png')
img_resize = cv2.resize(img, (224, 224))
img_rgb = cv2.cvtColor(img_resize, cv2.COLOR_BGR2RGB)
img_normalized = img_rgb.astype(np.float32) / 255.0
img_chw = np.transpose(img_normalized, (2, 0, 1))
img_tensor = np.expand_dims(img_chw, axis=0)
img_tensor_torch = torch.from_numpy(img_tensor).to(device)

# PyTorch 추론 시간
model.eval().to("cpu")
start = time.time()
with torch.no_grad():
    _ = model(img_tensor_torch.cpu())
end = time.time()
print(f"PyTorch 추론 시간: {end - start:.4f}초")

# ONNX Runtime 추론 시간
start = time.time()
_ = session.run(None, {"input": img_tensor})
end = time.time()
print(f"ONNX Runtime 추론 시간: {end - start:.4f}초")


## 실습